In [1]:
### Make notebook wider:

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [2]:

from pathlib import Path, PurePosixPath
from os.path import abspath
from loguru import logger
import sys
import os
import numpy as np
import pickle


In [11]:
from rich.progress import (
    Progress,
    BarColumn,
    TextColumn,
    ProgressColumn,
)
from rich.text import Text
from datetime import timedelta

from myterial import (
    orange,
    teal_light,
    light_blue_light,
)


class TimeRemainingColumn(ProgressColumn):
    """Renders estimated time remaining."""

    _table_column = None
    # Only refresh twice a second to prevent jitter
    max_refresh = 0.5

    def render(self, task):
        """Show time remaining."""
        remaining = task.time_remaining
        if remaining is None:
            return Text("-:--:--", style=teal_light)
        remaining_delta = timedelta(seconds=int(remaining))
        return Text("remaining: " + str(remaining_delta), style=teal_light)


class TimeElapsedColumn(ProgressColumn):
    """Renders estimated time elapsed."""

    _table_column = None
    # Only refresh twice a second to prevent jitter
    max_refresh = 0.5

    def render(self, task):
        """Show time elapsed."""
        elapsed = task.elapsed
        if elapsed is None:
            return Text("-:--:--", style=light_blue_light)
        elapsed_delta = timedelta(seconds=int(elapsed))
        return Text("elapsed: " + str(elapsed_delta), style=light_blue_light)


class SpeedColumn(TextColumn):
    _renderable_cache = {}
    _table_column = None

    def __init__(self, *args):
        pass

    def render(self, task):
        if task.speed is None:
            return " "
        else:
            return f"{task.speed:.1f} steps/s"


COLUMNS = [
    BarColumn(bar_width=None),
    "Completed: ",
    TextColumn("[bold magenta]Completed {task.completed}/{task.total}"),
    "[progress.percentage]{task.percentage:>3.0f}%",
    "•",
    "Speed: ",
    SpeedColumn(),
    "•",
    "Remaining: ",
    TimeRemainingColumn(),
    "Elpsed: ",
    TimeElapsedColumn(),
]

progress = Progress(*COLUMNS, transient=False,)

progess_with_description = Progress(
    *([TextColumn("[bold salmon]Task: {task.description}")] + COLUMNS),
    transient=False,
)


def track(iterable, total=None, description="Working...", transient=False):
    """
        Spawns a progress bar to monitor the progress of a for loop over
        an iterable sequence with detailed information.

        Arguments:
            iterable: list or other iterable object
            total: int. Total length of iterable
            description: str. Text to preprend to the progress bar.

        Returs:
            elements of iterable
    """
    description = f"[{orange}]" + description
    columns = [description] + COLUMNS

    if total is None:
        try:
            total = len(iterable)
        except Exception:
            raise ValueError(
                "Could not get total from iterable, pass a total value."
            )

    track_progress = Progress(*columns, transient=transient)

    with track_progress:
        yield from track_progress.track(
            iterable, total=total, description=description,
        )

def files(folderpath, pattern="*"):
    """
        returns all files folders in a given folder matching a pattern
    """
    return [f for f in folderpath.glob(pattern) if f.is_file()]

def return_nested_paths(current_path):
    paths = []
    for item in os.listdir(current_path):
        paths = paths + [current_path + '\\' + item]
    return paths

def create_bash_files(NAME, video_files,bash_folder,winstor_folder,BASH_TEMPLATE,BASH_INSERT):

    logger.info(
        f"Found {len(video_files)} videos left to track. Generating bash files."
    )
    for index,video in enumerate(track(video_files)):
        #winstor_video_path = winstor_folder / "raw" / "video" / video.name
        winstor_video_path = winstor_folder / os.path.join(video.parts[-5],video.parts[-4],video.parts[-3], video.parts[-2], video.parts[-1]).replace("\\","/") 

        # winstor_video_path = winstor_folder / "dlc" / "videos" / video.name
        winstor_save_path = winstor_folder / "dlc" / "tracking" / os.path.join(video.parts[-4],video.parts[-3], video.parts[-2]).replace("\\","/") 

        bash_content = BASH_INSERT.replace(
            "VIDEO", str(winstor_video_path)
        ).replace("DEST", str(winstor_save_path)).replace("NUMBER",str(index+1))

        BASH_TEMPLATE = BASH_TEMPLATE + bash_content

        if not os.path.exists(os.path.join(r'Z:\projects\sequence_squad\data\video_data\\', 'dlc', 'tracking', video.parts[-4],video.parts[-3], video.parts[-2]).replace("\\","/") ):
            print('yes')
            os.makedirs(os.path.join(r'Z:\projects\sequence_squad\data\video_data\\', 'dlc', 'tracking', video.parts[-4],video.parts[-3], video.parts[-2]).replace("\\","/") )


        bash_path = bash_folder / (NAME + ".sh")
        with open(bash_path, "w") as fout:
            fout.write(BASH_TEMPLATE)

        #### Fedes dos2unix way that didnt work for me: 
        #     # dos2unix
        #     content, outsize = "", 0
        #     with open(bash_path, "rb") as infile:
        #         content = infile.read()    
        #     with open(bash_path, "wb") as output:
        #         for line in content.splitlines():
        #             outsize += len(line) + 1

        # replacement strings
        WINDOWS_LINE_ENDING = b'\r\n'
        UNIX_LINE_ENDING = b'\n'
        # relative or absolute file path, e.g.:
        file_path = bash_path
        with open(file_path, 'rb') as open_file:
            content = open_file.read()

        content = content.replace(WINDOWS_LINE_ENDING, UNIX_LINE_ENDING)
        with open(file_path, 'wb') as open_file:
            open_file.write(content)


In [4]:
# create_bash_files(above_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\above_video_files"),PurePosixPath(r"/nfs/winstor/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE,BASH_INSERT)

In [16]:
Animals = ['EJT262']

videos_path = r"Z:\projects\sequence_squad\data\video_data\raw_flycap\\"


above_video_files = []
back_video_files = []
side_video_files = []
bottom_video_files = []


for animal in Animals:
    current_path = videos_path + animal 

    session_paths = return_nested_paths(current_path)
    for session_path in session_paths:
        experiment_segments = return_nested_paths(session_path)
        for experiment_segment in experiment_segments:
            if any(os.scandir(experiment_segment)): # if not empty 
                video_files = files(Path(experiment_segment), "*.avi")
                for item in video_files:
                    if 'above' in item.name:
                        above_video_files.append(item)
                    if 'back' in item.name:
                        back_video_files.append(item)
                    if 'side' in item.name:
                        side_video_files.append(item)
                    if 'bottom' in item.name:
                        bottom_video_files.append(item)
                        
# # prepare bash files
# BASH_TEMPLATE = """#! /bin/bash

# #SBATCH -p gpu # partition (queue)
# #SBATCH -N 1   # number of nodes
# #SBATCH --mem 40G # memory pool for all cores
# #SBATCH --gres=gpu:gtx1080
# #SBATCH -n 1
# #SBATCH -t 5-0:0 # time
# #SBATCH	-o out.out
# #SBATCH -e err.err
# #SBATCH --mail-user=emmett.thompson.14@ucl.ac.uk
# #SBATCH --mail-type=END,FAIL

# echo "loading conda env"
# module load miniconda
# module load cuda/11.5

# source activate DEEPLABCUT
# export DLClight=True

# echo "running tracking"
# python dlc_on_hpc.py '/nfs/winstor/sjones/projects/sequence_squad/data/video_data/dlc/trained_networks/sleep-tracking_above-view-sequence-2022-04-05/config.yaml' '/VIDEO' '/DEST'
# """



# # prepare bash files
# BASH_TEMPLATE = """#! /bin/bash

# #SBATCH -p gpu # partition (queue)
# #SBATCH -N 10   # number of nodes
# #SBATCH --mem 50G # memory pool for all cores
# #SBATCH --gres=gpu:1
# #SBATCH -n 1
# #SBATCH -t 5-0:0 # time
# #SBATCH	-o out.out
# #SBATCH -e err.err
# #SBATCH --mail-user=emmett.thompson.14@ucl.ac.uk
# #SBATCH --mail-type=END,FAIL

# echo "loading conda env"
# source activate DEEPLABCUT
# module load miniconda
# module load cuda/11.6
# module list

# echo "running tracking"
# python dlc_on_hpc.py '/nfs/winstor/sjones/projects/sequence_squad/data/video_data/dlc/trained_networks/sleep-tracking_above-view-sequence-2022-04-05/config.yaml' '/VIDEO' '/DEST'
# """



# prepare bash files
BASH_TEMPLATE = """#! /bin/bash

"""

# prepare bash files

#  REMEMBER TO CHANGE THE PATH TO THE NETWORK HERE!!!!

##Above
BASH_INSERT = """
echo "running tracking NUMBER" 
python dlc_on_hpc.py '/ceph/sjones/projects/sequence_squad/data/video_data/dlc/trained_networks/sleep-tracking_above-view-sequence-2022-04-05/config.yaml' '/VIDEO' '/DEST'
echo "DONE"
"""

create_bash_files('Above',above_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\\"),PurePosixPath(r"/ceph/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE,BASH_INSERT)

##back
BASH_INSERT = """
echo "running tracking NUMBER" 
python dlc_on_hpc.py '/ceph/sjones/projects/sequence_squad/data/video_data/dlc/trained_networks/Yellow-floor-task-tracking-backview-sequence-2023-04-09/config.yaml' '/VIDEO' '/DEST'
echo "DONE"
"""

create_bash_files('Back',back_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\\"),PurePosixPath(r"/ceph/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE,BASH_INSERT)

##side
BASH_INSERT = """
echo "running tracking NUMBER" 
python dlc_on_hpc.py '/ceph/sjones/projects/sequence_squad/data/video_data/dlc/trained_networks/task-tracking_sideview-sequence-2022-04-15/config.yaml' '/VIDEO' '/DEST'
echo "DONE"
"""

create_bash_files('Side',side_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\\"),PurePosixPath(r"/ceph/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE,BASH_INSERT)


#backports
BASH_INSERT = """
echo "running tracking NUMBER" 
python dlc_on_hpc.py '/ceph/sjones/projects/sequence_squad/data/video_data/dlc/trained_networks/port-tracking_back_view-sequence-2022-05-02/config.yaml' '/VIDEO' '/DEST'
echo "DONE"
"""

create_bash_files('Back_Ports',back_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\\"),PurePosixPath(r"/ceph/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE,BASH_INSERT)

# side ports
BASH_INSERT = """
echo "running tracking NUMBER" 
python dlc_on_hpc.py '/ceph/sjones/projects/sequence_squad/data/video_data/dlc/trained_networks/port-tracking_side-view-sequence-2022-04-29/config.yaml' '/VIDEO' '/DEST'
echo "DONE"
"""

create_bash_files('Side_Ports',side_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\\"),PurePosixPath(r"/ceph/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE,BASH_INSERT)







# create_bash_files('Side_Ports',back_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\\"),PurePosixPath(r"/nfs/winstor/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE,BASH_INSERT)
# create_bash_files('Side_Ports',side_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\\"),PurePosixPath(r"/nfs/winstor/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE,BASH_INSERT)

# task-tracking_bottomview-sequence-2022-04-25/config.yaml
# port-tracking_side-view-sequence-2022-04-29/config.yaml
# port-tracking_back_view-sequence-2022-05-02/config.yaml
#port-tracking_back_view-sequence-2022-05-02/config.yaml

# create_bash_files('Bottom',bottom_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\\"),PurePosixPath(r"/nfs/winstor/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE,BASH_INSERT)
# create_bash_files('Side_Ports',side_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\\"),PurePosixPath(r"/nfs/winstor/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE,BASH_INSERT)
# create_bash_files('Back_Ports',back_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\\"),PurePosixPath(r"/nfs/winstor/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE,BASH_INSERT)

# create_bash_files(above_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\above_video_files"),PurePosixPath(r"/nfs/winstor/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE,BASH_INSERT)
# create_bash_files(back_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\back_video_files"),PurePosixPath(r"/nfs/winstor/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE)
# create_bash_files(side_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\side_video_files"),PurePosixPath(r"/nfs/winstor/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE)
# create_bash_files(bottom_video_files,Path(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\bottom_video_files"),PurePosixPath(r"/nfs/winstor/sjones/projects/sequence_squad/data/video_data"),BASH_TEMPLATE)



2023-06-02 18:16:42.237 | INFO     | __main__:create_bash_files:131 - Found 12 videos left to track. Generating bash files.


Output()

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


2023-06-02 18:16:43.851 | INFO     | __main__:create_bash_files:131 - Found 7 videos left to track. Generating bash files.


Output()

...

Output()

2023-06-02 18:16:45.570 | INFO     | __main__:create_bash_files:131 - Found 7 videos left to track. Generating bash files.


Output()

2023-06-02 18:16:46.636 | INFO     | __main__:create_bash_files:131 - Found 7 videos left to track. Generating bash files.


Output()

In [7]:
above_video_files

[WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT268/010523/1_presleep/above_2023-05-01T14_02_46.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT268/010523/3_postsleep/above_2023-05-01T16_00_39.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT268/020523/1_presleep/above_2023-05-02T11_17_12.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT268/020523/3_postsleep/above_2023-05-02T13_17_47.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT268/030523/1_presleep/above_2023-05-03T14_18_25.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT268/030523/3_postsleep/above_2023-05-03T16_31_45.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT268/050523/1_presleep/above_2023-05-05T12_42_18.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT268/050523/3_postsleep/above_2023-05-05T14_3

In [8]:
len(bottom_video_files)

0

In [6]:
logger.info(
    f"Found {len(video_files)} videos left to track. Generating bash files."
)
for video in track(video_files):
    #winstor_video_path = winstor_folder / "raw" / "video" / video.name
    winstor_video_path = winstor_folder / os.path.join(video.parts[-5],video.parts[-4],video.parts[-3], video.parts[-2], video.parts[-1]).replace("\\","/") 

    # winstor_video_path = winstor_folder / "dlc" / "videos" / video.name
    winstor_save_path = winstor_folder / "dlc" / "tracking" / os.path.join(video.parts[-4],video.parts[-3], video.parts[-2]).replace("\\","/") 

    bash_content = BASH_INSERT.replace(
        "VIDEO", str(winstor_video_path)
    ).replace("DEST", str(winstor_save_path))
    
    BASH_TEMPLATE = BASH_TEMPLATE + bash_content

    if not os.path.exists(os.path.join(r'Z:\projects\sequence_squad\data\video_data\\', 'dlc', 'tracking', video.parts[-4],video.parts[-3], video.parts[-2]).replace("\\","/") ):
        print('yes')
        os.makedirs(os.path.join(r'Z:\projects\sequence_squad\data\video_data\\', 'dlc', 'tracking', video.parts[-4],video.parts[-3], video.parts[-2]).replace("\\","/") )

        
    bash_path = bash_folder / (NAME + ".sh")
    with open(bash_path, "w") as fout:
        fout.write(BASH_TEMPLATE)

    #### Fedes dos2unix way that didnt work for me: 
    #     # dos2unix
    #     content, outsize = "", 0
    #     with open(bash_path, "rb") as infile:
    #         content = infile.read()    
    #     with open(bash_path, "wb") as output:
    #         for line in content.splitlines():
    #             outsize += len(line) + 1

    # replacement strings
    WINDOWS_LINE_ENDING = b'\r\n'
    UNIX_LINE_ENDING = b'\n'
    # relative or absolute file path, e.g.:
    file_path = bash_path
    with open(file_path, 'rb') as open_file:
        content = open_file.read()

    content = content.replace(WINDOWS_LINE_ENDING, UNIX_LINE_ENDING)
    with open(file_path, 'wb') as open_file:
        open_file.write(content)

2023-05-24 15:51:29.344 | INFO     | __main__:<module>:1 - Found 3 videos left to track. Generating bash files.


Output()

NameError: name 'winstor_folder' is not defined

In [118]:
above_video_files

[WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT136/091121/1_presleep/above_2021-11-09T14_48_59.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT136/101121/1_presleep/above_2021-11-10T14_10_13.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT136/101121/3_postsleep/above_2021-11-10T16_16_46.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT136/111121/1_presleep/above_2021-11-11T14_18_47.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT136/111121/3_postsleep/above_2021-11-11T16_33_43.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT136/121121/1_presleep/above_2021-11-12T15_51_16.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT136/121121/3_postsleep/above_2021-11-12T17_38_46.avi'),
 WindowsPath('Z:/projects/sequence_squad/data/video_data/raw_flycap/EJT136_implant2/231121/1_presleep/above_2021-11

In [111]:
BASH_TEMPLATE + BASH_INSERT

'#! /bin/bash\n\n\necho "running tracking"\npython dlc_on_hpc.py \'/nfs/winstor/sjones/projects/sequence_squad/data/video_data/dlc/trained_networks/sleep-tracking_above-view-sequence-2022-04-05/config.yaml\' \'/VIDEO\' \'/DEST\'\n'

In [109]:
BASH_INSERT

'\necho "running tracking"\npython dlc_on_hpc.py \'/nfs/winstor/sjones/projects/sequence_squad/data/video_data/dlc/trained_networks/sleep-tracking_above-view-sequence-2022-04-05/config.yaml\' \'/VIDEO\' \'/DEST\'\n'

In [24]:
### save out paths to make life easier in the future (when sorting tracked videos)

path1 = PurePosixPath(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\above_video_files_paths.txt")
path2 = PurePosixPath(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\side_video_files_paths.txt")
path3 = PurePosixPath(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\back_video_files_paths.txt")
path4 = PurePosixPath(r"Z:\projects\sequence_squad\data\video_data\dlc\hpc_analysis\hpc_to_analyse\bottom_video_files_paths.txt")

with open(path1, "wb") as fp:   #Pickling
    pickle.dump(above_video_files,fp)

with open(path2, "wb") as fp:   #Pickling
    pickle.dump(above_video_files,fp)
    
with open(path3, "wb") as fp:   #Pickling
    pickle.dump(above_video_files,fp)
    
with open(path4, "wb") as fp:   #Pickling
    pickle.dump(above_video_files,fp)

In [250]:
#### to open again in the future: #

with open(path1, "rb") as fp:   # Unpickling
    relaoded_above_video_files = pickle.load(fp)